In [ ]:
# Drone Delivery Debrecen with No-Fly Zones

import pandas as pd
from shapely.geometry import Polygon, Point
import folium
import random
import googlemaps
import json

# Load Google Maps API key from a protected file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
GOOGLE_MAPS_API_KEY = config['GOOGLE_MAPS_API_KEY']

DEBRECEN_POLYGON_MAP_COORDINATES_PATH = 'coordinates/debrecen_coordinates_v1.csv'
NO_FLY_ZONE_COORDINATES_PATHS = [
    'coordinates/airport_coordinates_v1.csv',
    'coordinates/clinics_coordinates_v1.csv',
    'coordinates/kenezi_coordinates_v1.csv',
    'coordinates/mainsquare_coordinates_v1.csv',
    'coordinates/military_coordinates_v1.csv',
    'coordinates/railwaystation_coordinates_v1.csv',
    'coordinates/stadium_coordinates_v1.csv'
]
NUMBER_OF_SIMULATED_ORDERS = 10

FORUM_COORDINATE_X = 47.5326511
FORUM_COORDINATE_Y = 21.6287677

PLOT_ROUTES_ON_MAP = False

# Load Debrecen polygon coordinates
data = pd.read_csv(DEBRECEN_POLYGON_MAP_COORDINATES_PATH)
coordinates = list(zip(data['Latitude'], data['Longitude']))
debrecen_polygon = Polygon(coordinates)

# Load no-fly zones
no_fly_zones = []
for path in NO_FLY_ZONE_COORDINATES_PATHS:
    no_fly_data = pd.read_csv(path)
    no_fly_coordinates = list(zip(no_fly_data['Latitude'], no_fly_data['Longitude']))
    no_fly_zones.append(Polygon(no_fly_coordinates))

# Generate the base map
center_lat = data['Latitude'].mean()
center_lon = data['Longitude'].mean()
map_debrecen = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Add Debrecen polygon to the map
folium.Polygon(
    locations=coordinates,
    color='blue',
    fill=True,
    fill_color='lightblue',
    fill_opacity=0.5,
    weight=2
).add_to(map_debrecen)

# Add no-fly zones to the map
for no_fly_zone in no_fly_zones:
    folium.Polygon(
        locations=list(no_fly_zone.exterior.coords),
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        weight=2
    ).add_to(map_debrecen)

# Generate random points inside Debrecen polygon excluding no-fly zones
min_x, min_y, max_x, max_y = debrecen_polygon.bounds
random_points_inside_polygon = []

while len(random_points_inside_polygon) < NUMBER_OF_SIMULATED_ORDERS:
    random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
    if debrecen_polygon.contains(random_point) and not any(zone.contains(random_point) for zone in no_fly_zones):
        random_points_inside_polygon.append((random_point.x, random_point.y))

# Add random points to the map
for point in random_points_inside_polygon:
    folium.CircleMarker(
        location=point,
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
    ).add_to(map_debrecen)

# Mark Fórum Debrecen on the map
forum_point = Point(FORUM_COORDINATE_X, FORUM_COORDINATE_Y)
forum_point_data = (forum_point.x, forum_point.y)
folium.CircleMarker(
    location=forum_point_data,
    radius=8,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.9,
).add_to(map_debrecen)

# Initialize Google Maps API client
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Calculate routes and durations for each destination
results = []
for idx, destination in enumerate(random_points_inside_polygon):
    modes = ['driving', 'walking', 'bicycling']
    route_info = {'Destination': idx + 1, 'Coordinates': destination}

    for mode in modes:
        directions = gmaps.directions(
            origin=forum_point_data,
            destination=destination,
            mode=mode
        )

        if directions:
            leg = directions[0]["legs"][0]
            route_info[f'{mode}_duration'] = leg["duration"]["text"]
            route_info[f'{mode}_distance'] = leg["distance"]["text"]
        else:
            route_info[f'{mode}_duration'] = 'N/A'
            route_info[f'{mode}_distance'] = 'N/A'

    results.append(route_info)

# Plot driving routes on the map if enabled
if PLOT_ROUTES_ON_MAP:
    map_plot = folium.Map(location=[FORUM_COORDINATE_X, FORUM_COORDINATE_Y], zoom_start=13)

    # Add Fórum Debrecen marker
    folium.Marker(location=forum_point_data, popup="Start Point", icon=folium.Icon(color="blue")).add_to(map_plot)

    # Add destinations and driving routes
    for result in results:
        folium.Marker(
            location=result['Coordinates'],
            popup=f"Destination {result['Destination']}",
            icon=folium.Icon(color="red")
        ).add_to(map_plot)

        # Plot driving route
        directions = gmaps.directions(
            origin=forum_point_data,
            destination=result['Coordinates'],
            mode='driving'
        )

        if directions:
            steps = directions[0]['legs'][0]['steps']
            route_coordinates = [(step['start_location']['lat'], step['start_location']['lng']) for step in steps]
            route_coordinates.append(result['Coordinates'])
            folium.PolyLine(route_coordinates, color='green', weight=2.5, opacity=0.8).add_to(map_plot)

    map_plot.save("driving_routes_map.html")

# Save results to a DataFrame
results_df = pd.DataFrame(results)

# Save results, map and starting points
#results_df.to_csv("delivery_data.csv", sep=";")
#map_debrecen.save("delivery_data_map.html")

In [12]:
# %% Calculate drone routes using recursive safe routing to avoid drawing crossings into no-fly zones

from shapely.geometry import LineString, Point

def get_detour_route(zone, entry, exit):
    """
    Computes a detour route along the boundary of a zone between the entry and exit points.
    It works by inserting the entry and exit points into the polygon's coordinate list (if needed)
    and then returns the shorter of the two possible routes along the boundary.
    """
    coords = list(zone.exterior.coords)
    
    # Helper: insert a point into the boundary if it lies on a segment.
    def insert_point(coords, pt):
        new_coords = []
        for i in range(len(coords) - 1):
            seg = LineString([coords[i], coords[i+1]])
            # If the point lies on the segment (using a small tolerance)
            if seg.distance(pt) < 1e-8:
                new_coords.append(coords[i])
                if (pt.x, pt.y) != coords[i]:
                    new_coords.append((pt.x, pt.y))
            else:
                new_coords.append(coords[i])
        new_coords.append(coords[-1])
        if (pt.x, pt.y) not in new_coords:
            new_coords.append((pt.x, pt.y))
        return new_coords

    # Insert the entry and exit points into the boundary coordinates.
    new_coords = insert_point(coords, entry)
    new_coords = insert_point(new_coords, exit)
    
    # Find the indices of the entry and exit points.
    try:
        idx_entry = new_coords.index((entry.x, entry.y))
    except ValueError:
        idx_entry = 0
    try:
        idx_exit = new_coords.index((exit.x, exit.y))
    except ValueError:
        idx_exit = 0
    
    # Candidate 1: traverse from idx_entry to idx_exit along the list.
    if idx_entry <= idx_exit:
        candidate1 = new_coords[idx_entry:idx_exit+1]
    else:
        candidate1 = new_coords[idx_entry:] + new_coords[:idx_exit+1]
    
    # Candidate 2: the complementary route.
    if idx_exit <= idx_entry:
        candidate2 = new_coords[idx_exit:idx_entry+1]
    else:
        candidate2 = new_coords[idx_exit:] + new_coords[:idx_entry+1]
    
    candidate1_line = LineString(candidate1)
    candidate2_line = LineString(candidate2)
    
    if candidate1_line.length <= candidate2_line.length:
        return candidate1
    else:
        return candidate2

def compute_safe_route(start, end, zones):
    """
    Recursively computes a safe route from start to end that avoids crossing any no-fly zones.
    If a straight-line segment from start to end intersects a zone, it computes a detour along the zone's boundary
    between the entry and exit intersection points, and then recursively ensures that the segments before and after
    the detour are also safe.
    
    Parameters:
        start (tuple): (latitude, longitude) coordinates of the starting point.
        end (tuple): (latitude, longitude) coordinates of the ending point.
        zones (list): List of Polygon objects representing no-fly zones.
    
    Returns:
        A list of (latitude, longitude) coordinate tuples forming a safe route.
    """
    start_pt = Point(start)
    end_pt = Point(end)
    direct_line = LineString([start, end])
    
    for zone in zones:
        if direct_line.intersects(zone):
            # Get intersection points with the zone boundary.
            inter_geom = direct_line.intersection(zone.boundary)
            if inter_geom.is_empty:
                continue
            if inter_geom.geom_type == 'Point':
                pts = [inter_geom]
            elif inter_geom.geom_type == 'MultiPoint':
                pts = list(inter_geom.geoms)
            else:
                pts = []
            if len(pts) < 2:
                continue
            # Order intersections by distance from the start.
            pts = sorted(pts, key=lambda p: start_pt.distance(p))
            entry_pt = pts[0]
            exit_pt = pts[-1]
            # Compute a detour route along the zone boundary.
            detour_coords = get_detour_route(zone, entry_pt, exit_pt)
            # Reverse the detour coordinates so that the order becomes reversed.
            # For example: start, dt1, dt2, dt3, finish becomes start, dt3, dt2, dt1, finish.
            detour_coords = list(reversed(detour_coords))
            # Recursively ensure that the segments before and after the detour are safe.
            route_before = compute_safe_route(start, (entry_pt.x, entry_pt.y), zones)
            route_after  = compute_safe_route((exit_pt.x, exit_pt.y), end, zones)
            # Stitch the safe routes together, avoiding duplicate junction points.
            safe_route = route_before[:-1] + detour_coords + route_after[1:]
            return safe_route
    # If no zones are intersected, the direct route is safe.
    return [start, end]

# Compute safe routes for each destination.
drone_routes = []
for dest in random_points_inside_polygon:
    route = compute_safe_route(forum_point_data, dest, no_fly_zones)
    drone_routes.append(route)
    print(route)

# Plot the computed safe drone routes on the existing map.
for route in drone_routes:
    folium.PolyLine(route, color='purple', weight=3, opacity=0.8).add_to(map_debrecen)

# Save the updated map.
map_debrecen.save("drone_routes_map.html")

# Optionally, display the routes in a DataFrame for inspection.
drone_routes_df = pd.DataFrame({
    'Destination': list(range(1, len(drone_routes) + 1)),
    'Route': drone_routes
})
drone_routes_df


[(47.5326511, 21.6287677), (47.53899026091646, 21.62852789239161)]
[(47.5326511, 21.6287677), (47.54147920612253, 21.66555219976945)]
[(47.5326511, 21.6287677), (47.53180559870814, 21.624868869893298), (47.53288, 21.62562), (47.53341, 21.62541), (47.53294, 21.62386), (47.53294, 21.62386), (47.53172, 21.62365), (47.53155684706561, 21.62372181027042), (47.52508280787214, 21.593868303052407)]
[(47.5326511, 21.6287677), (47.52342938853508, 21.637166591740403), (47.52412, 21.63883), (47.52412, 21.63883), (47.52372, 21.63963), (47.5227626988421, 21.637773795140458), (47.5062680833272, 21.652796656193544)]
[(47.5326511, 21.6287677), (47.554203822434374, 21.600013263359614)]
[(47.5326511, 21.6287677), (47.52003452130571, 21.680417243116395)]
[(47.5326511, 21.6287677), (47.53264514784201, 21.625455811323178), (47.53288, 21.62562), (47.53341, 21.62541), (47.53294, 21.62386), (47.53294, 21.62386), (47.5326421877047, 21.623808737227858), (47.532608268940805, 21.604935721709285), (47.53168, 21.6051

,Destination,Route
0,1,"[(47.5326511, 21.6287677), (47.53899026091646,..."
1,2,"[(47.5326511, 21.6287677), (47.54147920612253,..."
2,3,"[(47.5326511, 21.6287677), (47.53180559870814,..."
3,4,"[(47.5326511, 21.6287677), (47.52342938853508,..."
4,5,"[(47.5326511, 21.6287677), (47.554203822434374..."
...,...,...
95,96,"[(47.5326511, 21.6287677), (47.53241633569084,..."
96,97,"[(47.5326511, 21.6287677), (47.49446216524447,..."
97,98,"[(47.5326511, 21.6287677), (47.538229583713964..."
98,99,"[(47.5326511, 21.6287677), (47.54635548458449,..."


Todo list
- költségkalkuláció kiszállítási módokhoz
- drónos útvonaltervezés a no-fly zone-ok kikerülésével
  - drónos kiszállítás költségkalkuláció
  - drónos kiszállítás időkalkuláció
- nagy adathalmaz legenerálása
- modell alkotás
- előrejelző szkript megírása